In [459]:
import pandas as pd
import numpy as np
from pycm import *

In [461]:
## FUNCTIONS

# function for getting all the text in lower case
def lower (x):
    return x.lower()

# remove punctuation
def remove (x):
    no_punct = ""
    for char in x:
            if char in '''1234567890qwertyuiopasdfghjklzxcvbnmQWERTYUIOPASDFGHJKLZXCVBNM ''':
                    no_punct = no_punct + char
    return no_punct


# find the position of the targets and the patterns
def find_position(x,str):
    return str.find(x)

# check if a phrase is in the sentence
def check(x, str):
    return x in str

# return the position of a phrase/word in a sentence
def pos_of_phrase(string, substring):
    substring_length = len(substring)    
    def recurse(locations_found, start):
        location = string.find(substring, start)
        if location != -1:
            return recurse(locations_found + [location], location+substring_length)
        else:
            return locations_found

    return recurse([], 0)

# return the minimum distance of all phrase/word from the target disease
def get_diff_of_pos(column,pos,str):
    name = classes.columns[column]
    my_col = classes[name]
    x = [k for k,v in enumerate(my_col) if v == True]
    
    candidate = []
    distances = []
    for i in range(len(x)):
        word = phrases[classes.columns[column]][x[i]]
        
        for i in range(len(pos_of_phrase(str, word))):
            distances.append( abs(pos -   pos_of_phrase(str, word)[i]) )

    return min(distances)
    
#  make all the annotations consistant   
def rep (text):
    a = text.replace('No consensus','IDK')
    b = a.replace('Contraindication','contraindication')
    c = b.replace('Effect','effect')
    d = c.replace('Indication: Treatment', 'treatment')
    e = d.replace('Indication: Symptomatic Relief', 'relief')
    f = e.replace('no consensus', 'IDK')
    return f

def rep2(text):
    a = text.replace('idk', 'IDK')
    return a


In [462]:
# Frames of rules

my_phrases = { 'effect': 
["hypersensitivity reactions",
"associated with the risk of",
"to the risk of",
"a high risk for",
"a high risk of",
"high incidence of", 
"higher incidence of", 
" cause ",
" causes ",
"symptoms occure",
"teratogenic",
"site reaction",
"the risk of development",
"is associated with a risk of",
"symptoms of the poisoning",
"symptoms of poisoning" 
],

 'contraindication': 
["not administrated to",
"contraindicated in",
"contraindicatedin",
"should not be used",
"is contraindication for",
"is contraindication when",
"must not be used for",
"do not administer",
"should not initiate",
"not be administered to",
"do not initiate patients",
"contraindication for",
"should not be given",
"do not use"],
              
 'relief': 
["relief of the signs",
"relief of signs",
"relief of symptoms",
"relief of the symptoms",
"help",
"helps",
"relief of signs and symptoms of",
"reduction of symptoms of",
"treatment of the symptoms of",
"for the relief",
"management of the signs and symptoms of"], 

 'treatment':
[" indicated for the treatment of",
" indicated in the management of",
" indicated for the management of",
"for the management of",
"management of",
" indicated for the maintenance of remission", 
"or the treatment of",
"in the treatment of",
" indicated as",
" indicated in",
"be effective",
"active treatment of",
" indicated for",
"treatment of",
" indicated as an adjunct",
" indicated for use in the treatment of", 
" indicated for the intermittent treatment", 
" indicated to reduce the rate of",
" indicated for the rapid control",
" indicated for the control",
"reduce the risk of",
" indicated as adjunctive treatment",
"for the treatment of",
" indicated as an adjunct"]
           }

phrases = pd.DataFrame({ key:pd.Series(value) for key, value in my_phrases.items() })


effect = { 'phrases': 
["hypersensitivity reactions",
"associated with the risk of",
"to the risk of",
"a high risk for",
"a high risk of",
"high incidence of", 
"higher incidence of", 
" cause ",
" causes ",
"symptoms occure",
"teratogenic",
"site reaction",
"the risk of development",
"is associated with a risk of",
"symptoms of the poisoning",
"symptoms of poisoning" 
]}
effect = pd.DataFrame(effect)

 
contraindication = {'phrases': 
["not administrated to",
"contraindicated in",
"contraindicatedin",
"should not be used",
"is contraindication for",
"is contraindication when",
"must not be used for",
"do not administer",
"should not initiate",
"not be administered to",
"do not initiate patients",
"contraindication for",
"should not be given",
"do not use"]}
contraindication = pd.DataFrame(contraindication)
              
relief = {'phrases': 
["relief of the signs",
"relief of signs",
"relief of symptoms",
"relief of the symptoms",
"help",
"helps",
"relief of signs and symptoms of",
"reduction of symptoms of",
"treatment of the symptoms of",
"for the relief",
"management of the signs and symptoms of"]} 
relief = pd.DataFrame(relief)

treatment = {'phrases':
[" indicated for the treatment of",
" indicated in the management of",
" indicated for the management of",
"for the management of",
"management of",
" indicated for the maintenance of remission", 
"or the treatment of",
"in the treatment of",
" indicated as",
" indicated in",
"be effective",
"active treatment of",
" indicated for",
"treatment of",
" indicated as an adjunct",
" indicated for use in the treatment of", 
" indicated for the intermittent treatment", 
" indicated to reduce the rate of",
" indicated for the rapid control",
" indicated for the control",
"reduce the risk of",
" indicated as adjunctive treatment",
"for the treatment of",
" indicated as an adjunct"]}
treatment = pd.DataFrame(treatment)

# Experts

In [ ]:
# Define the dataset

df= pd.read_csv('experts_dailymed.csv')
print('length of experts set:', len(df))
df.sample()

In [ ]:
# Check if there is something missing and drop it to avoid errors
print(df[df['DOID'].isnull()]['DOID'])
print(df[df['DBID'].isnull()]['DBID'])
print(df[df['text'].isnull()]['text'])

In [465]:
df = df[['text','disease', 'DOID',  'drug', 'DBID', 'experts', 'expert1', 'expert2', 'expert3']]
df['label']= 'label' # create this column where the label from the model will come

text = df['text']
disease = df['disease']
drug = df['drug']
answer = df['experts']
df.sample()

text = text.apply(lower)
text = text.apply(remove)

disese = disease.apply(lower)
drug = drug.apply(lower)

df['expert1'] = df['expert1'].apply(rep)
df['expert2'] = df['expert2'].apply(rep)
df['expert3'] = df['expert3'].apply(rep)
df['experts'] = df['experts'].apply(rep)

In [466]:
# clean them

data = {
'text': text,
'drug':drug, 
'DOID': df['DOID'],
'disease':disease, 
'DBID': df['DBID'],
'label': 'label' , 
'expert1': df['expert1'],
'expert2': df['expert2'],
'expert3': df['expert3'],
'answer': df['experts']
} 
df1 = pd.DataFrame(data)

df1.sample()

,text,drug,DOID,disease,DBID,label,expert1,expert2,expert3,answer
224,hypersensitivity 41pregnancy unless used for t...,goserelin,DOID_162,cancer,DB00014,label,treatment,inconsistent,treatment,treatment


## Compare with the cases that all experts aggree

In [467]:
df2 = df1.loc[(df1['expert1'] == df1['expert2'])  & (df1['expert1'] == df1['expert3'])].reset_index()

In [ ]:
for i in range(len(df2)):
    str = df2['text'][i]
    

# Find which phrases appear in the sentence
    class_effect = effect['phrases'].apply(lambda num : check(num,str))
    class_contraindication = contraindication['phrases'].apply(lambda num : check(num,str))
    class_relief = relief['phrases'].apply(lambda num : check(num,str))
    class_treatment = treatment['phrases'].apply(lambda num : check(num,str))

# Create a dataset from them
    classes = {
    'effect': class_effect,
    'contraindication': class_contraindication,
    'relief': class_relief,
    'treatment':class_treatment 
    }

    classes = pd.DataFrame(classes)

    a = class_effect.index[class_effect == True].tolist()
    b = class_contraindication.index[class_contraindication == True].tolist()
    c = class_relief.index[class_relief == True].tolist()
    d = class_treatment.index[class_treatment == True].tolist()


    # Check if you have unique class and label based on that 
    if len(a)!=0 and len(b)==len(c)==len(d)==0:
        #print('effect')
        df2['label'][i] = 'effect'
    elif len(b)!=0 and len(a)==len(c)==len(d)==0:
        #print('contraindication')
        df2['label'][i] =  'contraindication'
    elif len(c)!=0 and len(a)==len(b)==len(d)==0:
        #print('relief')
        df2['label'][i] = 'relief'
    elif len(d)!=0 and len(a)==len(b)==len(c)==0:
        #print('treatment')
        df2['label'][i] =  'treatment'
    elif len(a)==len(b)==len(c)==len(d)==0:
        #print('IDK')
        df2['label'][i] = 'IDK'
    else: 
        pos1 = str.find(df2['disease'][i])
        test_list = [len(a), len(b), len(c), len(d)] 
        res = [idx for idx, val in enumerate(test_list) if val != 0]
        my_List = [get_diff_of_pos(res[j], pos1, str) for j in range(len(res))]
        idx = my_List.index(min(my_List))
        classes.columns[res[idx]]
        #print(classes.columns[res[idx]])
        df2['label'][i] =  classes.columns[res[idx]]


In [469]:
df2['accurate'] = (df2['label'] == df2['answer'])
mistakes = df2['accurate'].index[ df2['accurate']== False].tolist()
accuracy = (len(df2)-len(mistakes))/len(df2)
print('number of missclassified instances:', len(mistakes))
print('accuracy:', accuracy)

number of missclassified instances: 13
accuracy: 0.912751677852349


### Compute the confusion matrix

In [470]:
from pycm import *

pred = df2['label'].tolist()
corr = df2 ['answer'].tolist()

cm = ConfusionMatrix(pred, corr)

confusion_matrix = pd.DataFrame(cm.table)
confusion_matrix.columns=['IDK', 'contraindication', 'effect', 'relief', 'treatment']
confusion_matrix.index = ['IDK', 'contraindication', 'effect', 'relief', 'treatment']
confusion_matrix

,IDK,contraindication,effect,relief,treatment
IDK,0,0,0,0,0
contraindication,5,33,6,0,0
effect,0,1,15,0,0
relief,0,0,0,22,0
treatment,0,1,0,0,66


In [ ]:
# Compute Accuracy
FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
PRE = (TP)/(TP+FP)
REC = (TP)/(TP+FN)
F1 = 2*(TP)/(2*TP+FP+FN)

print('---Accuracy---')
print(ACC)
print('---Precision---')
print(PRE)
print('---Recall---')
print(REC)
print('---F1-score---')
print(F1)

In [472]:
df2.sample()

,index,text,drug,DOID,disease,DBID,label,expert1,expert2,expert3,answer,accurate
22,48,zafirlukast tablets are indicated for the prop...,zafirlukast,DOID_2841,asthma,DB00549,treatment,treatment,treatment,treatment,treatment,True


## Compare with the cases that all experts aggree

In [473]:
df3 = df1[['text', 'drug', 'DOID', 'disease', 'DBID', 'label', 'answer']]
df3.sample()

,text,drug,DOID,disease,DBID,label,answer
122,carefully consider the potential benefits and ...,naproxen,DOID_7147,ankylosing spondylitis,DB00788,label,relief


In [474]:
for i in range(len(df3)):
    str = df3['text'][i]
    

# Find which phrases appear in the sentence
    class_effect = effect['phrases'].apply(lambda num : check(num,str))
    class_contraindication = contraindication['phrases'].apply(lambda num : check(num,str))
    class_relief = relief['phrases'].apply(lambda num : check(num,str))
    class_treatment = treatment['phrases'].apply(lambda num : check(num,str))

# Create a dataset from them
    classes = {
    'effect': class_effect,
    'contraindication': class_contraindication,
    'relief': class_relief,
    'treatment':class_treatment 
    }

    classes = pd.DataFrame(classes)

    a = class_effect.index[class_effect == True].tolist()
    b = class_contraindication.index[class_contraindication == True].tolist()
    c = class_relief.index[class_relief == True].tolist()
    d = class_treatment.index[class_treatment == True].tolist()


    # Check if you have unique class and label based on that 
    if len(a)!=0 and len(b)==len(c)==len(d)==0:
        #print('effect')
        df3['label'][i] = 'effect'
    elif len(b)!=0 and len(a)==len(c)==len(d)==0:
        #print('contraindication')
        df3['label'][i] =  'contraindication'
    elif len(c)!=0 and len(a)==len(b)==len(d)==0:
        #print('relief')
        df3['label'][i] = 'relief'
    elif len(d)!=0 and len(a)==len(b)==len(c)==0:
        #print('treatment')
        df3['label'][i] =  'treatment'
    elif len(a)==len(b)==len(c)==len(d)==0:
        #print('IDK')
        df3['label'][i] = 'IDK'
    else: 
        pos1 = str.find(df3['disease'][i])
        test_list = [len(a), len(b), len(c), len(d)] 
        res = [idx for idx, val in enumerate(test_list) if val != 0]
        my_List = [get_diff_of_pos(res[j], pos1, str) for j in range(len(res))]
        idx = my_List.index(min(my_List))
        classes.columns[res[idx]]
        #print(classes.columns[res[idx]])
        df3['label'][i] =  classes.columns[res[idx]]

df3.sample()

,text,drug,DOID,disease,DBID,label,answer
289,durlaza is indicated toreduce the risk of deat...,aspirin,DOID_5844,myocardial infarction,DB00945,treatment,treatment


In [ ]:
df3['accurate'] = (df3['label'] == df3['answer'])
mistakes = df3['accurate'].index[ df3['accurate']== False].tolist()
accuracy = (len(df3)-len(mistakes))/len(df3)
print('number of missclassified instances:', len(mistakes))
print('accuracy:', accuracy)

In [ ]:
df3 ['answer']= df3 ['answer'].apply(rep)

In [476]:
pred = df3['label'].tolist()
corr = df3 ['answer'].tolist()

cm = ConfusionMatrix(pred, corr)
confusion_matrix = pd.DataFrame(cm.table)
confusion_matrix.columns=['idk', 'contraindication', 'effect', 'relief', 'treatment']
confusion_matrix.index = ['idk', 'contraindication', 'effect', 'relief', 'treatment']
confusion_matrix

,idk,contraindication,effect,relief,treatment
idk,5,12,2,0,9
contraindication,10,77,19,0,1
effect,6,14,22,0,5
relief,0,3,0,30,16
treatment,1,2,9,1,114


In [477]:
# Compute Accuracy
FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
PRE = (TP)/(TP+FP)
REC = (TP)/(TP+FN)
F1 = 2*(TP)/(2*TP+FP+FN)

In [ ]:
print('---Accuracy---')
print(ACC)
print('---Precision---')
print(PRE)
print('---Recall---')
print(REC)
print('---F1-score---')
print(F1)

## Compare with at least one expert

In [ ]:
df6 = df1[['text', 'drug', 'DOID', 'disease', 'DBID', 'expert1', 'expert2', 'expert3','label']]
df6['expert1'] = df6['expert1'].apply(rep2)
df6['expert2'] = df6['expert2'].apply(rep2)
df6['expert3'] = df6['expert3'].apply(rep2)
df6['at_least'] = df6['expert3'].apply(rep2) # here will go the True/False of prediction being the same  with at least one expert

In [479]:
for i in range(len(df6)):
    str = df6['text'][i]
    

# Find which phrases appear in the sentence
    class_effect = effect['phrases'].apply(lambda num : check(num,str))
    class_contraindication = contraindication['phrases'].apply(lambda num : check(num,str))
    class_relief = relief['phrases'].apply(lambda num : check(num,str))
    class_treatment = treatment['phrases'].apply(lambda num : check(num,str))

# Create a dataset from them
    classes = {
    'effect': class_effect,
    'contraindication': class_contraindication,
    'relief': class_relief,
    'treatment':class_treatment 
    }

    classes = pd.DataFrame(classes)

    a = class_effect.index[class_effect == True].tolist()
    b = class_contraindication.index[class_contraindication == True].tolist()
    c = class_relief.index[class_relief == True].tolist()
    d = class_treatment.index[class_treatment == True].tolist()


    # Check if you have unique class and label based on that 
    if len(a)!=0 and len(b)==len(c)==len(d)==0:
        #print('effect')
        df6['label'][i] = 'effect'
    elif len(b)!=0 and len(a)==len(c)==len(d)==0:
        #print('contraindication')
        df6['label'][i] =  'contraindication'
    elif len(c)!=0 and len(a)==len(b)==len(d)==0:
        #print('relief')
        df6['label'][i] = 'relief'
    elif len(d)!=0 and len(a)==len(b)==len(c)==0:
        #print('treatment')
        df6['label'][i] =  'treatment'
    elif len(a)==len(b)==len(c)==len(d)==0:
        #print('IDK')
        df6['label'][i] = 'IDK'
    else: 
        pos1 = str.find(df6['disease'][i])
        test_list = [len(a), len(b), len(c), len(d)] 
        res = [idx for idx, val in enumerate(test_list) if val != 0]
        my_List = [get_diff_of_pos(res[j], pos1, str) for j in range(len(res))]
        idx = my_List.index(min(my_List))
        classes.columns[res[idx]]
        #print(classes.columns[res[idx]])
        df6['label'][i] =  classes.columns[res[idx]]

In [480]:
for i in range(len(df6)):
    if df6['label'][i] == df6['expert1'][i]:
        df6['at_least'][i] = True
    elif df6['label'][i] == df6['expert2'][i]:
        df6['at_least'][i] = True
    elif df['label'][i] == df6['expert3'][i]:
        df6['at_least'][i] =True
    else:
        df6['at_least'][i] = False  

In [481]:
df6.sample()

,text,drug,DOID,disease,DBID,expert1,expert2,expert3,label,at_least
296,in the treatment of parkinsonism rigidity and ...,atropine,DOID_13250,diarrhea,DB00572,effect,effect,effect,effect,True


In [482]:
mistakes = df6['at_least'].index[ df6['at_least']== False].tolist()
accuracy = (len(df6)-len(mistakes))/len(df6)
print('number of missclassified instances:', len(mistakes))
print('accuracy:', accuracy)

number of missclassified instances: 77
accuracy: 0.7849162011173184


# Crowd

In [484]:
df4= pd.read_csv('crowd_dailymed.csv')
print('length of experts set:', len(df4))

length of experts set: 4046


In [486]:
data = {
'text': df4['text'],
'drug': df4['drug'], 
'DOID': df4['DOID'],
'disease': df4['disease'], 
'DBID': df4['DBID'],
'label': 'label' , 
'crowd': df4['relation']
} 

df5 = pd.DataFrame(data)

df5.sample()

,text,drug,DOID,disease,DBID,label,crowd
4031,['LINZESS is indicated in adults for the treat...,Dicyclomine,DOID_9778,irritable bowel syndrome,DB08890,label,treatment


In [487]:
for i in range(len(df5)):
    str = df5['text'][i]
    
# Find which phrases appear in the sentence
    class_effect = effect['phrases'].apply(lambda num : check(num,str))
    class_contraindication = contraindication['phrases'].apply(lambda num : check(num,str))
    class_relief = relief['phrases'].apply(lambda num : check(num,str))
    class_treatment = treatment['phrases'].apply(lambda num : check(num,str))

# Create a dataset from them
    classes = {
    'effect': class_effect,
    'contraindication': class_contraindication,
    'relief': class_relief,
    'treatment':class_treatment 
    }

    classes = pd.DataFrame(classes)

    a = class_effect.index[class_effect == True].tolist()
    b = class_contraindication.index[class_contraindication == True].tolist()
    c = class_relief.index[class_relief == True].tolist()
    d = class_treatment.index[class_treatment == True].tolist()


    # Check if you have unique class and label based on that 
    if len(a)!=0 and len(b)==len(c)==len(d)==0:
        #print('effect')
        df5['label'][i] = 'effect'
    elif len(b)!=0 and len(a)==len(c)==len(d)==0:
        #print('contraindication')
        df5['label'][i] =  'contraindication'
    elif len(c)!=0 and len(a)==len(b)==len(d)==0:
        #print('relief')
        df5['label'][i] = 'relief'
    elif len(d)!=0 and len(a)==len(b)==len(c)==0:
        #print('treatment')
        df5['label'][i] =  'treatment'
    elif len(a)==len(b)==len(c)==len(d)==0:
        #print('IDK')
        df5['label'][i] = 'IDK'
    else: 
        pos1 = str.find(df5['disease'][i])
        test_list = [len(a), len(b), len(c), len(d)] 
        res = [idx for idx, val in enumerate(test_list) if val != 0]
        my_List = [get_diff_of_pos(res[j], pos1, str) for j in range(len(res))]
        idx = my_List.index(min(my_List))
        classes.columns[res[idx]]
        #print(classes.columns[res[idx]])
        df5['label'][i] =  classes.columns[res[idx]]



In [488]:
df5['accurate'] = (df5['label'] == df5['crowd'])
mistakes = df5['accurate'].index[ df5['accurate']== False].tolist()
accuracy = (len(df5)-len(mistakes))/len(df5)
print('number of missclassified instances:', len(mistakes))
print('accuracy:', accuracy)

number of missclassified instances: 1519
accuracy: 0.6245674740484429


In [489]:
pred = df5['label'].tolist()
corr = df5 ['crowd'].tolist()

cm = ConfusionMatrix(pred, corr)
confusion_matrix = pd.DataFrame(cm.table)
confusion_matrix.columns=['IDK', 'contraindication', 'effect', 'relief', 'treatment']
confusion_matrix.index = ['IDK', 'contraindication', 'effect', 'relief', 'treatment']
confusion_matrix

,IDK,contraindication,effect,relief,treatment
IDK,87,277,35,1,67
contraindication,40,293,42,0,79
effect,42,111,63,0,42
relief,53,198,27,78,147
treatment,101,161,69,27,2006


In [490]:
# Compute Accuracy
FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
TP = np.diag(confusion_matrix)
TN = confusion_matrix.values.sum() - (FP + FN + TP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
PRE = (TP)/(TP+FP)
REC = (TP)/(TP+FN)
F1 = 2*(TP)/(2*TP+FP+FN)

In [ ]:
print('---Accuracy---')
print(ACC)
print('---Precision---')
print(PRE)
print('---Recall---')
print(REC)
print('---F1-score---')
print(F1)